In [1]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# read from melodyData.txt
with open('generated/melodyData.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


Number of parts:11554


In [40]:
sequence_length = 1

X = []
Y = []
# given data_encoded, generate training data by looping 
for i in range(len(data_encoded[:100])):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + sequence_length])
        Y.append(data_encoded[i][j + sequence_length])

X_onehot = []
for seq in X:
    onehot = np.zeros((len(seq), len(encodings)))
    for note_index in range(len(seq)):
        onehot[note_index][seq[note_index]] = 1
    X_onehot.append(onehot)
X = X_onehot


# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#print (X_train) # each element of the list is a 2d array

In [41]:
#sequence_length = 1

# X_train should contain sequences from parts 1-100
# X_test should contain the answers to said sequences
# Y_train should contains sequences from parts 201-210
# Y_test should contain the answers to said sequences

def generate_data(data_encoded, encodings, sequence_length=1):

    X = []
    Y = []
    for i in range(len(data_encoded)):
        for j in range(len(data_encoded[i]) - sequence_length):
            X.append(data_encoded[i][j:j + sequence_length])
            #print (data_encoded[i][j + sequence_length])
            Y.append(data_encoded[i][j + sequence_length])
    
    X_onehot = []

    """
X_onehot = []
for seq in X:
    onehot = np.zeros((len(seq), len(encodings)))
    for note_index in range(len(seq)):
        onehot[note_index][seq[note_index]] = 1
    X_onehot.append(onehot)
X = X_onehot
    """


    for seq in X:
        onehot = np.zeros(len(encodings))
        onehot[seq[0]] = 1
        X_onehot.append(onehot)
        
    X = X_onehot

    return X, Y

X_train, Y_train = generate_data(data_encoded[:100], encodings)
X_test, Y_test = generate_data(data_encoded[200:205], encodings)

print (len(X_train))

14326


In [43]:
#split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# T    tf.keras.layers.Input(shape=(sequence_length, 1)), X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([

    #KEEP STATEFUL = TRUE

    tf.keras.Input(batch_shape=(1, 1, len(encodings))),
    #tf.keras.layers.Input(shape=(sequence_length, 1)),
    #tf.keras.layers.Flatten(),
    # tf.keras.layers.LSTM(128, return_sequences=True),
    # tf.keras.layers.LSTM(64),

    tf.keras.layers.LSTM(128, stateful=True),  # stateful=True, batch size 1
    #tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dense(len(encodings), activation='softmax')

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='softplus'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=15)


Epoch 1/15


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 1159), dtype=float32). Expected shape (1, 1, 1159), but input has incompatible shape (None, 1159)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 1159), dtype=float32)
  • training=True
  • mask=None

In [73]:
loss, accuracy = model.evaluate(np.array(X_test), np.array(Y_test), verbose=1)
print(f'Loss: {loss}, Accuracy: {accuracy}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1303 - loss: 17.2807  
Loss: 21.392276763916016, Accuracy: 0.1330561339855194


In [75]:
import copy
# given a sequence, generate the next note (up to ten times)
seq = copy.deepcopy(X_test[4])
total_seq = seq[:]
for i in range(10):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    # append to sequence the one-hot encoding of pred
    one_hot = np.zeros((len(encodings)))
    one_hot[pred] = 1
    total_seq = np.append(total_seq, [one_hot], axis=0)
    # set sequence to be the last ten values of total_seq
    seq = total_seq[-10:]

    #total_seq.append(pred)
    # total seq is a numpy array, can't just append things to it

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}
    

for note in total_seq:
    pitch, dur = decodings[np.argmax(note)]
    print (mapping[pitch % 12], dur)

# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
G 0.33
G 0.25
C 0.75
D 0.25
C 0.75
C 0.25
C 0.75
A# 0.25
C 0.75
D# 0.25
C 0.25
A# 0.50
C 0.50
C 0.25
C 0.25
A# 0.50
C 0.50
F# 0.50
G# 0.50
D 0.50


In [2]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split